# 데이터 불러오기

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from IPython.display import set_matplotlib_formats

# 레티나 디스플레이 설정
set_matplotlib_formats('retina')

In [17]:
onlinesales_info = pd.read_csv('../RAW/Onlinesales_info.csv')
tax_info = pd.read_csv('../RAW/Tax_info.csv')
discount_info = pd.read_csv('../RAW/Discount_info.csv')
customer_info = pd.read_csv('../RAW/Customer_info.csv')
# 데이터 불러오기

In [88]:
def type_desc(df):
    obj_col = df.select_dtypes(include=['object']).columns.to_list()
    num_col = df.select_dtypes(include=['float','int']).columns.to_list()
    result_dict = {
        'obj' : df[obj_col].describe(),
        'num' : df[num_col].describe()
    }
    return result_dict

# 기술통계량 도출 함수 정의

# 데이터 이해

## onlinesales 데이터프레임

In [97]:
online_desc = type_desc(onlinesales_info)
onlinesales_info.isna().sum()

고객ID      0
거래ID      0
거래날짜      0
제품ID      0
제품카테고리    0
수량        0
평균금액      0
배송료       0
쿠폰상태      0
dtype: int64

In [98]:
online_desc['obj']

,고객ID,거래ID,거래날짜,제품ID,제품카테고리,쿠폰상태
count,52924,52924,52924,52924,52924,52924
unique,1468,25061,365,1145,20,3
top,USER_0118,Transaction_12261,2019-11-27,Product_0981,Apparel,Clicked
freq,695,35,335,3511,18126,26926


In [99]:
online_desc['num']

,수량,평균금액,배송료
count,52924.000000,52924.000000,52924.000000
mean,4.497638,52.237646,10.517630
std,20.104711,64.006882,19.475613
min,1.000000,0.390000,0.000000
25%,1.000000,5.700000,6.000000
50%,1.000000,16.990000,6.000000
75%,2.000000,102.130000,6.500000
max,900.000000,355.740000,521.360000


## tax 데이터프레임

In [107]:
tax_desc = type_desc(tax_info)
tax_info.isna().sum()

제품카테고리    0
GST       0
dtype: int64

In [103]:
tax_desc['obj']

,제품카테고리
count,20
unique,20
top,Nest-USA
freq,1


In [104]:
tax_desc['num']

,GST
count,20.000000
mean,0.116500
std,0.052443
min,0.050000
25%,0.087500
50%,0.100000
75%,0.180000
max,0.180000


## discount 데이터프레임

In [106]:
discount_desc = type_desc(discount_info)
discount_info.isna().sum()

월         0
제품카테고리    0
쿠폰코드      0
할인율       0
dtype: int64

In [108]:
discount_desc['obj']

,월,제품카테고리,쿠폰코드
count,204,204,204
unique,12,17,48
top,Jan,Apparel,EXTRA10
freq,17,12,8


In [109]:
discount_desc['num']

,할인율
count,204.000000
mean,20.000000
std,8.185052
min,10.000000
25%,10.000000
50%,20.000000
75%,30.000000
max,30.000000


## customer 데이터프레임

In [110]:
customer_desc = type_desc(customer_info)
customer_info.isna().sum()

고객ID    0
성별      0
고객지역    0
가입기간    0
dtype: int64

In [111]:
customer_desc['obj']

,고객ID,성별,고객지역
count,1468,1468,1468
unique,1468,2,5
top,USER_1358,여,California
freq,1,934,464


In [112]:
customer_desc['num']

,가입기간
count,1468.000000
mean,25.912125
std,13.959667
min,2.000000
25%,14.000000
50%,26.000000
75%,38.000000
max,50.000000


---

# 데이터 병합 및 전처리 과정

**1. 고객 ID 기준 병합**
    : customer_info, onlinesales_info
    
**2. 제품 카테고리 기준 병합**
    : tax_info, discount_info

**3. 제품 카테고리, 월 기준 병합**
    : 1 병합 결과, 2 병합 결과

In [163]:
id_merge = pd.merge(onlinesales_info,customer_info,on='고객ID')
# 1 병합
cat_merge = pd.merge(tax_info,discount_info,on='제품카테고리')
# 2 병합

In [164]:
month_mapping = {
    1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun',
    7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'
}
id_merge['월'] = pd.to_datetime(id_merge['거래날짜']).dt.month
id_merge['월'] = id_merge['월'].map(lambda x: month_mapping[x])
# 월 데이터를 병합을 위해 변환 

In [174]:
merge_df = pd.merge(id_merge,cat_merge,on=['제품카테고리','월'],how='left')
# 최종 병합본 생성

In [176]:
# merge_df.to_csv('../PROCESSED/merge_df.csv',index=False)
# 병합 데이터프레임 저장